In [9]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer
from transformers import BartForConditionalGeneration, AutoTokenizer
import numpy as np
import pandas as pd
from tqdm import tqdm

In [10]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")



def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_labels)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

In [20]:


classifier = RobertaForSequenceClassification.from_pretrained("liamcripwell/ctrl44-clf",cache_dir='/ssd_scratch/cvit/aparna/classification')
tokenizer1 = AutoTokenizer.from_pretrained("liamcripwell/ctrl44-clf",cache_dir='/ssd_scratch/cvit/aparna/classification_tokenizer')

model = BartForConditionalGeneration.from_pretrained("liamcripwell/ctrl44-simp",cache_dir='/ssd_scratch/cvit/aparna/paraphrase')
tokenizer2 = AutoTokenizer.from_pretrained("liamcripwell/ctrl44-simp",cache_dir='/ssd_scratch/cvit/aparna/paraphrase_tokenizer')


test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()

metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    inputs = tokenizer1(text, return_tensors="pt")
    inpu.extend([text]*10)
    with torch.no_grad():
        logits = classifier(**inputs).logits
        predicted_class_id = logits.argmax().item()
        predicted_class_name = classifier.config.id2label[predicted_class_id]
        text = predicted_class_name + " " + text
        inputs1 = tokenizer2(text, return_tensors="pt")
        beam_outputs = model.generate(**inputs1,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=10,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
    c = []
    for x, beam in enumerate(beam_outputs):
    #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer2.decode(beam, skip_special_tokens=True))
    cands.extend(c)

    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]
    #print(i)
    if i%100==0:
        metric = metrics_func(eval_arg)
        metrics.append(metric)
        print(metric)
        
print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  0%|          | 1/10000 [00:32<91:29:50, 32.94s/it]

10 10 10
3
(34.61004551647818, 39.15192624769725)


  1%|          | 73/10000 [31:24<196:58:36, 71.43s/it]

In [12]:
from transformers import RobertaForSequenceClassification, AutoTokenizer

model = RobertaForSequenceClassification.from_pretrained("liamcripwell/ctrl44-clf")
tokenizer = AutoTokenizer.from_pretrained("liamcripwell/ctrl44-clf")

text = "Barack Hussein Obama II is an American politician who served as the 44th president of the United States from 2009 to 2017."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
  logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
predicted_class_name = model.config.id2label[predicted_class_id]

In [16]:

print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


5020 5010 5010
5020 5010 5010
3


ValueError: Mismatch in the number of sources (5020) and predictions (5010)